In [204]:
import requests
from bs4 import BeautifulSoup
import re
import json
import collections
import pandas as pd
from fuzzywuzzy import fuzz, process
import math
pd.set_option('display.max_columns', None)


In [254]:
html = requests.get("https://www.fantasyfootballscout.co.uk/team-news/").text
soup = BeautifulSoup(html, "html.parser")


In [255]:
raw = soup.find_all("div", {"class": re.compile('formation.*')})
teams = [x.text.split('Next')[0] for x in soup.find_all('header')[-20:]]
lineups = [x.text.split('  ')[1:] for x in raw]

In [256]:
xi = {team: lineup for team, lineup in zip(teams, lineups)}

In [257]:
html = requests.get("https://understat.com/league/EPL").text
soup = BeautifulSoup(html, "html.parser")

In [258]:
pscript = soup.find_all('script')[3].string
sidx = pscript.index("parse") + 7
eidx = pscript.index(");") - 1

In [259]:
playerdata = json.loads(pscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [260]:
teamscript = soup.find_all('script')[2].string
sidx = teamscript.index("parse") + 7
eidx = teamscript.index(")") - 1

In [261]:
teamdata = json.loads(teamscript[sidx:eidx].encode('utf8').decode('unicode_escape'))

In [262]:
fd = collections.defaultdict(list)

for d in playerdata:
    for key, value in d.items():
        fd[key].append(value)
playersxg = pd.DataFrame(fd)

In [321]:
teamsxg = pd.DataFrame(teamdata.values())
teamsxg = teamsxg.explode("history")
h = teamsxg.pop("history")
teamsxg = pd.concat([teamsxg.reset_index(drop=True), pd.DataFrame(h.tolist())], axis=1)
teamsxg = teamsxg.infer_objects()
teamsxg = teamsxg.groupby(['title', 'h_a']).sum().reset_index()
teamsxg['matches'] = teamsxg['wins'] + teamsxg['draws'] + teamsxg['loses']
teamsxg = teamsxg.rename(columns = {'title':'team','scored':'goals_scored','missed':'goals_conceded'})
teamsxg['goals_conceded_90'] = teamsxg['goals_conceded']/teamsxg['matches']
teamsxg['xG_90'] = teamsxg['xG']/teamsxg['matches']
teamsxg['xGA_90'] = teamsxg['xGA']/teamsxg['matches']
aag = teamsxg.loc[teamsxg['h_a'] == 'a','xG_90'].mean()
ahg = teamsxg.loc[teamsxg['h_a'] == 'h','xG_90'].mean()
aaga = teamsxg.loc[teamsxg['h_a'] == 'a','xGA_90'].mean()
ahga = teamsxg.loc[teamsxg['h_a'] == 'h','xGA_90'].mean()
# teamsxg['xG_diff'] = teamsxg['xG_90'] - teamsxg['xGA_90']
teamsxg['xG_strength'] = teamsxg.apply(lambda row: xgstrength(row['h_a'], row['xG_90'], aag, ahg), axis = 1)
teamsxg['xGA_strength'] = teamsxg.apply(lambda row: xgstrength(row['h_a'], row['xGA_90'], aaga, ahga), axis = 1)

In [332]:
teams_xg_90['Arsenal']

1.6123195909090908

In [339]:
g = teamsxg.groupby(['team'], as_index=False).mean()
teams_xg_90 = dict(zip(g['team'],g['xG_90']))
teamsxg['xG_multiplier'] = teamsxg.apply(lambda row: row['xG_90']/teams_xg_90[row['team']], axis = 1)

In [341]:
teamsxg

,team,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,goals_scored,goals_conceded,xpts,wins,draws,loses,pts,npxGD,matches,goals_conceded_90,xG_90,xGA_90,xG_strength,xGA_strength,xG_multiplier
0,Arsenal,a,12.764230,18.599086,12.003060,17.076746,65,83,15,17,11.4984,4,1,5,13,-5.073686,10,1.700000,1.276423,1.859909,1.010631,1.187562,0.791669
1,Arsenal,h,21.430378,10.955202,19.036117,10.194032,100,56,18,8,22.8048,7,2,2,23,8.842085,11,0.727273,1.948216,0.995927,1.260730,0.776409,1.208331
2,Aston Villa,a,11.460010,16.591755,10.698841,14.197495,84,75,11,16,11.9803,4,0,7,12,-3.498654,11,1.454545,1.041819,1.508341,0.824879,0.963084,1.006417
3,Aston Villa,h,10.285343,14.146676,9.524174,12.624336,57,71,17,16,11.5630,4,2,4,14,-3.100162,10,1.600000,1.028534,1.414668,0.665585,1.102852,0.993583
4,Brentford,a,11.067635,17.865260,10.306495,17.865260,53,61,12,21,11.5491,2,4,5,10,-7.558765,11,1.909091,1.006149,1.624115,0.796636,1.037006,0.810211
5,Brentford,h,17.730267,11.941248,16.207924,11.180115,78,76,14,17,20.7227,4,1,7,13,5.027809,12,1.416667,1.477522,0.995104,0.956134,0.775767,1.189789
6,Brighton,a,12.107475,14.311124,12.107475,12.788790,84,71,13,12,12.7073,3,7,1,16,-0.681315,11,1.090909,1.100680,1.301011,0.871483,0.830703,0.905019
7,Brighton,h,14.648812,11.543309,11.604150,10.782173,90,70,10,11,18.2624,3,5,3,14,0.821977,11,1.000000,1.331710,1.049392,0.861777,0.818089,1.094981
8,Burnley,a,8.385727,20.816751,8.385727,20.816751,56,106,8,19,6.5299,0,5,6,5,-12.431024,11,1.727273,0.762339,1.892432,0.603596,1.208328,0.734914
9,Burnley,h,9.186059,8.560492,9.186059,8.560492,30,51,8,8,9.5688,1,4,2,7,0.625567,7,1.142857,1.312294,1.222927,0.849212,0.953374,1.265086


In [316]:
test = {"Arsenal":1.27, "Aston Villa":1.63}

In [560]:
poisspdf(0, 1.27)

0.2808316217783798

In [319]:
for i in range(0,5):
    mt = []
    odd1 = poisspdf(i, 1.63)
    for j in range(0,5):
        mt.append(poisspdf(j, 1.27) * odd1)
    print(mt)

[0.055023220056407245, 0.06987948947163719, 0.044373475814489624, 0.018784771428133944, 0.005964164928432527]
[0.0896878486919438, 0.11390356783876862, 0.07232876557761808, 0.030619177427858326, 0.009721588833345018]
[0.0730955966839342, 0.09283140778859643, 0.05894794394575874, 0.024954629603704537, 0.00792309489917619]
[0.0397152741982709, 0.050438398231804044, 0.032028382877195574, 0.01355868208467946, 0.004304881561885729]
[0.016183974235795394, 0.020553647279460147, 0.013051566022457195, 0.00552516294950688, 0.0017542392364684343]


In [336]:
def calcmult(xg_90, team):
    return xg_90 / teams_xg_90[team]

In [280]:
def xgstrength(h_a, xg_90, aa, ah):
    if h_a == 'a':
        return xg_90/aa
    else:
        return xg_90/ah
    

In [304]:
def calcgoals(h_a, col1, col2, aah, ahg):
    if h_a == 'a':
        return col1 * col2 * aah
    else:
        return col1 * col2 * ahg

In [265]:
players.to_csv('players.csv', index = False)

In [373]:
players = pd.read_csv('players.csv')
players['name_team'] = players['player_name'] + ' ' + players['team_title']

In [542]:
p = list(filter(lambda x: ',' not in x, sorted(players.team_title.unique())))
x = sorted(list(xi.keys()))
d = {k:v for k, v in zip(x, p)}

In [268]:
lop = [i for j in xi.values() for i in j]


In [553]:
def getname(name):
    highest = process.extractOne(name,list(players.name_team))
    return highest[0]

In [572]:
list(players.player_name)

['Mohamed Salah', 'Diogo Jota', 'Jamie Vardy', 'Son Heung-Min', 'Michail Antonio', 'Sadio Mané', 'Cristiano Ronaldo', 'Emile Smith-Rowe', 'Raphinha', 'Emmanuel Dennis', 'Raheem Sterling', 'Bruno Fernandes', 'Neal Maupay', 'Bernardo Silva', 'Mason Mount', 'Conor Gallagher', 'Kevin De Bruyne', 'Callum Wilson', 'Riyad Mahrez', 'James Ward-Prowse', 'Ivan Toney', 'Jorginho', 'Jarrod Bowen', 'Maxwel Cornet', 'James Maddison', 'Bukayo Saka', 'Allan Saint-Maximin', 'Joshua King', 'Wilfried Zaha', 'Romelu Lukaku', 'Harry Kane', 'Demarai Gray', 'Pablo Fornals', 'Said Benrahma', 'Odsonne Edouard', 'Ismaila Sarr', 'Youri Tielemans', 'Phil Foden', 'Mason Greenwood', 'Teemu Pukki', 'Armando Broja', 'Ollie Watkins', 'Ilkay Gündogan', 'Pierre-Emerick Aubameyang', 'Roberto Firmino', 'Manuel Lanzini', 'Marcus Rashford', 'Christian Benteke', 'Danny Ings', 'Martin Odegaard', 'Raúl Jiménez', 'Richarlison', 'Leandro Trossard', 'Che Adams', 'Gabriel Martinelli', 'Reece James', 'Alexis Mac Allister', 'Jack Ha

In [573]:
def adjustxi(xi):
    for team, players in xi.items():
        for idx, player in enumerate(players):
            if player == 'S Armstrong':
                players[idx] = 'Stuart Armstrong'
            elif player == 'D Sanchez':
                players[idx] = 'David Sanchez'
            elif player == 'Wood':
                players[idx] = 'Chris Wood'
            elif player == 'Sanchez':
                players[idx] = 'Robert Sanchez'

In [385]:
def getxG(name):
    xg = players.loc[players['name_team'] == getname(name), "xG"].values[0]
    minutes = players.loc[players['name_team'] == getname(name), "time"].values[0]
    return xg/minutes*90

In [578]:
getxG("Robert Sanchez")

0.0

In [533]:
for team in d.keys():
    for i in xi[team]:
        print(f"{i} {getname(i + ' ' + d[team])} {team}")

Ramsdale Aaron Ramsdale Arsenal Arsenal
Tomiyasu Takehiro Tomiyasu Arsenal Arsenal
White Ben White Arsenal Arsenal
Gabriel Gabriel Arsenal Arsenal
Tierney Kieran Tierney Arsenal Arsenal
Sambi Lokonga Albert Sambi Lokonga Arsenal Arsenal
Saka Bukayo Saka Arsenal Arsenal
Odegaard Martin Odegaard Arsenal Arsenal
Smith-Rowe Emile Smith-Rowe Arsenal Arsenal
Gabriel Martinelli Gabriel Martinelli Arsenal Arsenal
Lacazette Alexandre Lacazette Arsenal Arsenal
Martinez Emiliano Martinez Aston Villa Aston Villa
Cash Matthew Cash Aston Villa Aston Villa
Hause Kortney Hause Aston Villa Aston Villa
Mings Tyrone Mings Aston Villa Aston Villa
Digne Lucas Digne Aston Villa,Everton Aston Villa
McGinn John McGinn Aston Villa Aston Villa
Douglas Luiz Douglas Luiz Aston Villa Aston Villa
Sanson Morgan Sanson Aston Villa Aston Villa
Buendía Emiliano Buendía Aston Villa Aston Villa
Jacob Ramsey Jacob Ramsey Aston Villa Aston Villa
Watkins Ollie Watkins Aston Villa Aston Villa
Lössl Jonas Lössl Brentford Bren

Winks Harry Winks Tottenham Tottenham Hotspur
Højbjerg Pierre-Emile Højbjerg Tottenham Tottenham Hotspur
Reguilón Sergio Reguilón Tottenham Tottenham Hotspur
Kane Harry Kane Tottenham Tottenham Hotspur
Lucas Moura Lucas Moura Tottenham Tottenham Hotspur
Foster Ben Foster Watford Watford
Femenía Kiko Femenía Watford Watford
Cathcart Craig Cathcart Watford Watford
Samir Samir Watford Watford
Kamara Hassane Kamara Watford Watford
Sissoko Moussa Sissoko Watford Watford
Kayembe Edo Kayembe Watford Watford
Cleverley Tom Cleverley Watford Watford
Dennis Emmanuel Dennis Watford Watford
King Joshua King Watford Watford
João Pedro João Pedro Watford Watford
Fabianski Lukasz Fabianski West Ham West Ham United
Coufal Vladimir Coufal West Ham West Ham United
Dawson Craig Dawson West Ham West Ham United
Diop Issa Diop West Ham West Ham United
Cresswell Aaron Cresswell West Ham West Ham United
Soucek Tomas Soucek West Ham West Ham United
Rice Declan Rice West Ham West Ham United
Bowen Jarrod Bowen We

In [535]:
namexg = {}
teamxg = {}
for team, plys in xi.items():
    xgsum = 0
    for ply in plys:
        plxg = getxG(ply + ' ' + d[team])
        xgsum = xgsum + plxg
        namexg[ply] = plxg
    teamxg[team] = xgsum

In [527]:
d

{'Arsenal': 'Arsenal', 'Aston Villa': 'Aston Villa', 'Brentford': 'Brentford', 'Brighton and Hove Albion': 'Brighton', 'Burnley': 'Burnley', 'Chelsea': 'Chelsea', 'Crystal Palace': 'Crystal Palace', 'Everton': 'Everton', 'Leeds United': 'Leeds', 'Leicester City': 'Leicester', 'Liverpool': 'Liverpool', 'Manchester City': 'Manchester City', 'Manchester United': 'Manchester United', 'Newcastle United': 'Newcastle United', 'Norwich City': 'Norwich', 'Southampton': 'Southampton', 'Tottenham Hotspur': 'Tottenham', 'Watford': 'Watford', 'West Ham United': 'West Ham', 'Wolverhampton Wanderers': 'Wolverhampton Wanderers'}

In [538]:
dict(sorted(namexg.items(), key=lambda item: item[1]))

{'Ramsdale': 0.0, 'Martinez': 0.0, 'Lössl': 0.0, 'Pope': 0.0, 'Pieters': 0.0, 'Kepa': 0.0, 'Butland': 0.0, 'Pickford': 0.0, 'Mykolenko': 0.0, 'Meslier': 0.0, 'Hjelde': 0.0, 'Schmeichel': 0.0, 'Alisson': 0.0, 'Ederson': 0.0, 'De Gea': 0.0, 'Dubravka': 0.0, 'Trippier': 0.0, 'Dummett': 0.0, 'Gunn': 0.0, 'Forster': 0.0, 'Lloris': 0.0, 'Winks': 0.0, 'Foster': 0.0, 'Kamara': 0.0, 'Fabianski': 0.0, 'José Sá': 0.0, 'Amartey': 0.0011278431905248625, 'Femenía': 0.002813890960413148, 'Hughes': 0.003210670130651754, 'Cathcart': 0.003401552047390285, 'Diop': 0.0035811460111290125, 'Mitchell': 0.003953377484895621, 'Williams': 0.004013674269853791, 'Sanson': 0.004831591875038345, 'Struijk': 0.005212923265336191, 'White': 0.006087445447014433, 'Choudhury': 0.008054929579385589, 'Allan': 0.00805650096122593, 'Veltman': 0.008356910344068104, 'Skipp': 0.010055566405635754, 'Konaté': 0.010169318744114458, 'Thomas': 0.010216686504049444, 'Walker': 0.010766490126506918, 'Andersen': 0.012833118229223478, 'C

In [547]:
players[players.team_title == "Brighton"]

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,name_team
12,3621,Neal Maupay,19,1405,7,6.036237,1,1.549384,36,15,4,0,F S,Brighton,6,5.275068,8.139677,2.082598,Neal Maupay Brighton
52,7698,Leandro Trossard,20,1664,4,4.082470,1,1.796554,49,25,1,0,F M S,Brighton,3,3.321301,7.015662,2.882910,Leandro Trossard Brighton
56,8379,Alexis Mac Allister,16,740,4,2.860665,2,1.126770,24,15,1,0,M S,Brighton,3,2.099496,4.538906,2.389454,Alexis Mac Allister Brighton
77,501,Danny Welbeck,8,370,2,1.044051,0,0.202304,10,3,0,0,F M S,Brighton,2,1.044051,1.319975,0.170015,Danny Welbeck Brighton
179,6047,Shane Duffy,14,1161,1,0.618664,0,0.060798,17,2,5,0,D S,Brighton,1,0.618664,2.630814,2.611065,Shane Duffy Brighton
193,7382,Dan Burn,12,935,1,0.944086,0,0.695415,6,2,3,0,D S,Brighton,1,0.944086,1.790943,1.790943,Dan Burn Brighton
196,7699,Adam Webster,12,893,1,0.188093,0,0.036906,4,1,4,0,D S,Brighton,1,0.188093,2.452004,2.434987,Adam Webster Brighton
209,9734,Enock Mwepu,11,637,1,1.595615,3,1.071449,10,5,1,0,M S,Brighton,1,1.595615,2.576522,1.522883,Enock Mwepu Brighton
218,239,Pascal Groß,14,996,0,1.433991,2,2.753167,12,30,0,0,D M S,Brighton,0,0.672823,4.271191,2.650129,Pascal Groß Brighton
223,486,Adam Lallana,18,1379,0,0.822218,1,0.912458,18,7,4,0,F M S,Brighton,0,0.822218,4.799247,4.236681,Adam Lallana Brighton


In [536]:
teamxg

{'Arsenal': 1.755493530220422, 'Aston Villa': 0.8217222469731091, 'Brentford': 1.2999098169508043, 'Brighton and Hove Albion': 1.5983945947984248, 'Burnley': 0.7403748107552841, 'Chelsea': 2.0334049014801803, 'Crystal Palace': 1.2550966334134985, 'Everton': 1.5674494984160225, 'Leeds United': 1.0955055754826648, 'Leicester City': 1.6913018505991966, 'Liverpool': 5.277702377548407, 'Manchester City': 2.477549345347949, 'Manchester United': 1.694095298009335, 'Newcastle United': 1.1915245553952498, 'Norwich City': 1.0867607191450088, 'Southampton': 1.3888341502401687, 'Tottenham Hotspur': 2.128355580697539, 'Watford': 1.0450107896678835, 'West Ham United': 1.6172323685851226, 'Wolverhampton Wanderers': 1.001068007728719}

In [169]:
def poisspdf(x, mu):
    return mu**x * math.e**-mu / math.factorial(x)

In [273]:
poissl = []
for xg in teamxg.values():
    row = []
    for i in range(0,7):
        row.append(poisspdf(i, xg))
    poissl.append(row)
    

In [274]:
pd.DataFrame(poissl, index = teamxg.keys())

,0,1,2,3,4,5,6
Arsenal,0.264680,0.351821,0.233827,0.103604,0.034428,0.009153,0.002028
Aston Villa,0.484872,0.350984,0.127034,0.030652,0.005547,0.000803,0.000097
Brentford,0.301027,0.361399,0.216940,0.086816,0.026057,0.006257,0.001252
Brighton and Hove Albion,0.317469,0.364256,0.208969,0.079922,0.022925,0.005261,0.001006
Burnley,0.591883,0.310411,0.081397,0.014229,0.001866,0.000196,0.000017
Chelsea,0.229922,0.337989,0.248424,0.121729,0.044736,0.013153,0.003222
Crystal Palace,0.367797,0.367879,0.183981,0.061341,0.015339,0.003068,0.000512
Everton,0.194474,0.318443,0.260718,0.142305,0.058254,0.019078,0.005207
Leeds United,0.417674,0.364652,0.159181,0.046324,0.010111,0.001765,0.000257
Leicester City,0.306189,0.362391,0.214454,0.084606,0.025034,0.005926,0.001169
